In [99]:
import pandas as pd
import pandasql as pds
import numpy as np
import re 
import os
import matplotlib
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
datapath = "d:/programming/GitHub/Projects/data/"
os.chdir(datapath)

In [109]:
df = pd.read_csv("carFeaturesAndMSRP.csv")
df

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11909,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,46120
11910,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,56670
11911,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,50620
11912,Acura,ZDX,2013,premium unleaded (recommended),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,50920


In [111]:
df.rename(columns = {col : col.replace(" ", "").replace("_", "") for col in df.columns}, inplace = True)
enginefueltypes = df.EngineFuelType.unique().tolist() 
df.replace(to_replace = {enginefueltypes[i]:i for i in range(len(enginefueltypes))}, inplace = True)
transmissiontypes = df.TransmissionType.unique().tolist()
df.replace(to_replace = {transmissiontypes[i]:i for i in range(len(transmissiontypes))}, inplace = True)
drivenwheels = df.DrivenWheels.unique().tolist()
df.replace(to_replace = {drivenwheels[i]:i for i in range(len(drivenwheels))}, inplace = True)
vehiclesize = df.VehicleSize.unique().tolist()
df.replace(to_replace = {vehiclesize[i]:i for i in range(len(vehiclesize))}, inplace = True)
vehiclestyle = df.VehicleStyle.unique().tolist()
df.replace(to_replace = {vehiclestyle[i]:i for i in range(len(vehiclestyle))}, inplace = True)
del df['MarketCategory']
df

,Make,Model,Year,EngineFuelType,EngineHP,EngineCylinders,TransmissionType,DrivenWheels,NumberofDoors,VehicleSize,VehicleStyle,highwayMPG,citympg,Popularity,MSRP
0,BMW,1 Series M,2011,0.0,335.0,6.0,0,0,2.0,0,0,26,19,3916,46135
1,BMW,1 Series,2011,0.0,300.0,6.0,0,0,2.0,0,1,28,19,3916,40650
2,BMW,1 Series,2011,0.0,300.0,6.0,0,0,2.0,0,0,28,20,3916,36350
3,BMW,1 Series,2011,0.0,230.0,6.0,0,0,2.0,0,0,28,18,3916,29450
4,BMW,1 Series,2011,0.0,230.0,6.0,0,0,2.0,0,1,28,18,3916,34500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11909,Acura,ZDX,2012,0.0,300.0,6.0,1,2,4.0,1,4,23,16,204,46120
11910,Acura,ZDX,2012,0.0,300.0,6.0,1,2,4.0,1,4,23,16,204,56670
11911,Acura,ZDX,2012,0.0,300.0,6.0,1,2,4.0,1,4,23,16,204,50620
11912,Acura,ZDX,2013,2.0,300.0,6.0,1,2,4.0,1,4,23,16,204,50920
